In [2]:
import warnings
warnings.filterwarnings(
    "ignore", ".*Trying to infer the `batch_size`.*"
)
warnings.filterwarnings(
    "ignore", ".*The dataloader,.*"
)

import torch
from torch import nn
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from src.datasets import VoxCelebDataModule
from src.losses import (
    AAMSoftmaxLoss, SubCenterAAMSoftmaxLoss, SoftmaxLoss
)
from src.models import (
    ResNet34, EfficientNetV2, build_efficientnetv2,
    Var_ECAPA, MHA_LAS, ResNet34SE
)
from src.resnetse import ResNetSE, SEBasicBlock, ResNetSEV2

seed_everything(42, workers=True)

Global seed set to 42


42

Better resnet34
Just add MHA and see


In [3]:
num_classes = 8
from_memory = True
spec_augment = True

dm3 = VoxCelebDataModule(
    batch_size=16,
    from_memory=from_memory,
    spec_augment=spec_augment,
    num_secs=3
    # num_workers=4,
    # data_dir="/home/gabriele/datasets/voxceleb1/"
)
dm6 = VoxCelebDataModule(
    batch_size=16,
    from_memory=from_memory,
    spec_augment=spec_augment,
    num_secs=6
    # num_workers=4,
    # data_dir="/home/gabriele/datasets/voxceleb1/"
)

# ResNet34

## Train (3 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet34(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=aams,
    optimizer="Adam",
    lr_scheduler=None
)
logger = TensorBoardLogger("tb_logs", name="resnet34")
trainer = Trainer(
    deterministic=True,
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm3)

## Test (3 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)
checkpoint_path = "checkpoints/resnet34/last.ckpt"
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512,
    scale=30,
    margin=0.2
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512
)
model = ResNet34(num_classes=num_classes, loss_func=aams) \
        .load_from_checkpoint(
            checkpoint_path, 
            num_classes=num_classes, 
            loss_func=aams
        )
logger = TensorBoardLogger("tb_logs", name="resnet34")
trainer = Trainer(
    deterministic=True,
    gpus=1,
    gradient_clip_val=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    resume_from_checkpoint=checkpoint_path
)
trainer.test(
    model, 
    dm,
    ckpt_path=checkpoint_path
)

# ResNet34-SE

## Train (3 secs)

In [4]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34se/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet34SE(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    lr_scheduler=None
)
logger = TensorBoardLogger(
    save_dir="tb_logs", 
    name="resnet34se",
    sub_dir="3secs"
)
trainer = Trainer(
    deterministic=True,
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm3)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Loading data for val: 100%|██████████| 29/29 [00:00<00:00, 60.60it/s]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name         | Type                    | Params
----------------------------------------------------------
0  | loss_func    | SubCenterAAMSoftmaxLoss | 6.1 K 
1  | train_acc    | Accuracy                | 0     
2  | val_acc      | Accuracy                | 0     
3  | train_f1     | F1Score                 | 0     
4  | val_f1       | F1Score                 | 0     
5  | instancenorm | InstanceNorm2d          | 0     
6  | conv1        | Conv2d                  | 3.1 K 
7  | bn1          | BatchNorm2d             | 128   
8  | gelu         | GELU                    | 0     
9  | maxpool      | MaxPool2d               | 0     
10 | conv2_x      | Sequential              | 225 K 
11 | conv3_x      | Sequential              | 1.1 M 
12 | conv4_x      | Sequential

Epoch 7:  14%|█▎        | 54/393 [00:06<00:41,  8.15it/s, loss=0.141, v_num=13, train_loss_step=0.149, train_acc_step=0.938, train_f1_step=0.908, val_loss_step=1.220, val_acc_step=0.846, val_f1_step=0.844, val_loss_epoch=1.480, val_acc_epoch=0.655, val_f1_epoch=0.613, train_loss_epoch=0.129, train_acc_epoch=0.959, train_f1_epoch=0.958]     

D:\Users\gabri\anaconda3\envs\tf_p3.9\lib\site-packages\pytorch_lightning\trainer\trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Epoch 7:  14%|█▎        | 54/393 [00:18<01:57,  2.88it/s, loss=0.141, v_num=13, train_loss_step=0.149, train_acc_step=0.938, train_f1_step=0.908, val_loss_step=1.220, val_acc_step=0.846, val_f1_step=0.844, val_loss_epoch=1.480, val_acc_epoch=0.655, val_f1_epoch=0.613, train_loss_epoch=0.129, train_acc_epoch=0.959, train_f1_epoch=0.958]

## Train (6 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34se/6secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet34SE(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    lr_scheduler=None
)
logger = TensorBoardLogger("tb_logs", name="resnet34se")
trainer = Trainer(
    deterministic=True,
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model=model, 
    datamodule=dm6,
    ckpt_path="checkpoints/resnet34se/last.ckpt"
)

# EfficientNetV2

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/efficientnetv2/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
model = build_efficientnetv2(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=softmax,
    optimizer="Adam",
    # lr=0.001,
    #lr_scheduler="ReducePlateau",
    #lr_scheduler_interval="epoch"
)

logger = TensorBoardLogger(
    "tb_logs", 
    name="efficientnetv2",
    sub_dir="3secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm3)

# MHA_LAS

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/MHA_LAS/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=30,
    margin=0.2
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = MHA_LAS(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    # lr_scheduler="CyclicLR"
)
logger = TensorBoardLogger(
    "tb_logs", 
    name="mha_las",
    sub_dir="3secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm3)

# VAR_ECAPA

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/var_ecapa/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512
)
model = Var_ECAPA(
    num_classes=num_classes, 
    loss_func=aams
)
logger = TensorBoardLogger(
    "tb_logs", 
    name="var_ecapa",
    sub_dir="3secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm3)

# ResNetSE

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnetse/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=30,
    margin=0.3
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
num_filters = [16, 32, 64, 128]
model = ResNetSE(
    SEBasicBlock,
    [3, 4, 6, 3],
    num_filters,
    nOut=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
)

logger = TensorBoardLogger(
    "tb_logs", 
    name="resnetse",
    sub_dir="3secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm3)

# ResNetSEV2

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnetsev2/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=30,
    margin=0.3
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
num_filters = [32, 64, 128, 256]
model = ResNetSEV2(
    SEBasicBlock,
    [3, 4, 6, 3],
    num_filters,
    nOut=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
)

logger = TensorBoardLogger(
    "tb_logs", 
    name="resnetsev2",
    sub_dir="3secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm3)